# Additional tools unnecassary for daily work

This notebook contains various solutions that were used for intermediate work with datasets. They can be useful for future product expansion, but not for current work.

In [5]:
from os import listdir
import pandas as pd
import dask.dataframe as dd
from tqdm.notebook import tqdm
tqdm.pandas()

In [6]:
df_meta = pd.read_feather(f'../DATASETS/IT_TEXTS/META/it_articles_ru.feather')
df_meta = df_meta.rename(columns={'index':'url'})
df_meta

,url,title,summary,views_num,post_time,parse_time,likes_num,favs_num,comments_num,source,fulltext_saved
0,https://tproger.ru/articles/7-prakticheskih-za...,7 практических заданий с собеседования на пози...,Для начинающего разработчика очень важно не то...,8642,2021-03-01 09:05:11,2021-04-19,13,10,10,tproger,True
1,https://tproger.ru/quiz/test-chto-mozhet-jeta-...,"Тест: что реально, а что создала нейросеть?",Сегодня нейронные сети используются в сельском...,4967,2021-02-26 19:39:50,2021-04-19,2,1,0,tproger,True
2,https://tproger.ru/articles/kak-najti-dejstvit...,Как найти действительно хороший курс по разраб...,Сразу хочется пошутить и предложить разработат...,1291,2021-02-26 17:29:00,2021-04-19,1,4,1,tproger,True
3,https://tproger.ru/articles/blackbox-skanery-v...,Blackbox-сканеры в процессе оценки безопасност...,Профиль задач quality engineer (QE) достаточно...,233,2021-02-26 15:16:46,2021-04-19,0,1,0,tproger,True
4,https://tproger.ru/articles/instrumenty-upravl...,Инструменты управления командой разработки на ...,В КРОК геораспределенная структура офисов разр...,589,2021-02-25 18:56:54,2021-04-19,3,3,0,tproger,True
...,...,...,...,...,...,...,...,...,...,...,...
337989,https://dev.by/news/haiv-prodzhekt-dfr-i-kgb,"Директора «Хайв Проджект» (MolaMola, Ulej) уве...","Директора «Хайв Проджект» (MolaMola.by, Ulej.b...",<NA>,2020-08-04 22:01:00,2021-04-26,<NA>,<NA>,0,devby,True
337990,https://dev.by/news/dmitriev-salary,"Штаб Дмитриева запустил сервис, который считае...",Кандидат в президенты Андрей Дмитриев сообщил ...,<NA>,2020-07-31 16:04:00,2021-04-26,<NA>,<NA>,0,devby,True
337991,https://dev.by/news/hacaton-idea-vote,Что придумали 200+ разработчиков на хакатоне п...,18-19 июля в Минске прошёл хакатон Social Tech...,<NA>,2020-07-21 17:16:00,2021-04-26,<NA>,<NA>,0,devby,True
337992,https://dev.by/news/udalennaya-rabota-kak-chas...,"История компании, которая уже 13 лет на удалёнке",Концепция удаленной работы возникла гораздо ра...,<NA>,2020-07-21 10:19:00,2021-04-26,<NA>,<NA>,0,devby,True


## Compile many tiny fulltext files into a single feather file:

In [19]:
def read_func(path, filename):
    with open(path + filename, 'r') as reader:
        return reader.read()


def many_to_one(name):
    path = f'../IT_TEXTS/DATASETS/{name}/'
    df = pd.DataFrame({'filename':listdir(path)})
    df['fulltext'] = df.filename.progress_apply(lambda filename: read_func(path, filename))
    df.to_feather(f'../IT_TEXTS/DATASETS/FULLTEXTS/{name}.feather',
                  compression='zstd')

## Check that url `fulltext` exists

In [23]:
df_meta = df_meta.drop(columns='filename')

In [54]:
def check_fulltext(url):
    if url in fulltexts_url_list:
        return 1
    else:
        return 0

def check()
    path = '../IT_TEXTS/DATASETS/FULLTEXTS/'
    df_filenames = listdir(path)
    df = pd.concat([pd.read_feather(f'{path}{name}') for name in df_filenames ])
    df = df.reset_index().drop(columns='index')
    fulltexts_url_list = df.filename.to_list()
    df_meta['fulltext_saved'] = df_meta.url.progress_apply(check_fulltext)
    df_meta.to_feather(f'../IT_TEXTS/DATASETS/META/it_articles_ru.feather',
                   compression='zstd')

# Correct date parsing

Sometimes the date turns out to be parsed incorrectly, for example 12.04 can read by `pandas.to_datetime()` as December 4, and not April 12, which does not correspond to the Russian writing tradition (to initially avoid this, you need to use the appropriate attribute, but sometimes this haso't be done). To improve accuracy we used the following function.

In [46]:
def correct_date(x):
    try:
        if x.post_time  > x.parse_time:
            x.post_time = x.post_time.replace(day = x.post_time.month,
                                month = x.post_time.day)
    except ValueError:
        pass
    return x

df_meta = df_meta.progress_apply(correct_date, axis=1)